This notebook is derived from the 🤗 notebook here: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling-tf.ipynb#scrollTo=QRTpmyCc3l_T

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 3.8 MB 28.6 MB/s 
     |████████████████████████████████| 6.5 MB 56.8 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 69.1 MB/s 
     |████████████████████████████████| 895 kB 61.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 26.9 MB/s 
     |████████████████████████████████| 134 kB 74.0 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 212 kB 69.9 MB/s 
     |████████████████████████████████| 127 kB 70.9 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 271 kB 77.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

Make sure your version of Transformers is at least 4.16.0 since some of the functionality we use was only introduced in that version.

In [ ]:
import transformers

print(transformers.__version__)
import math



4.17.0
Fri Mar 18 19:36:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 18 19:36:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Preparing the dataset

In [ ]:
from datasets import load_dataset


In [ ]:
from google.colab import drive  
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [ ]:
 datasets = load_dataset("csv", 
                         data_files={"train": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_20000_with_context.csv",
                                     "validation": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_2000_with_context_validation.csv",
                                     "test": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_2000_with_context_test.csv"})

Using custom data configuration default-37fd03610a554aa2


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'confidence', 'source', 'string'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'confidence', 'source', 'string'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'confidence', 'source', 'string'],
        num_rows: 2000
    })
})


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,Unnamed: 0,confidence,source,string
0,3811681,0.878031,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/O/show_0OBHX9ti3cAtHa3LArLCUH/73ruay7QPPVQpgYj07O9ZA.json,She also gave some to her husband who was with her and he ate it then the eyes of both of them were opened and they realized they were naked so they sewed fig leaves together and made coverings for themselves.
1,1551849,0.852056,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/I/show_0ihG4EK5Qz4JFDm3R6UYCG/6ED0dszyWHiTy0X8L7txEL.json,But then the main character whose name is Will McAvoy portrayed by actor Jeff Daniels.
2,1590174,0.869567,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/1/show_01Txd706SjsgvM0cm0UXuM/2Bve8rW9t6x484VWdxScnc.json,"I mean, they've got you."
3,1092753,0.861631,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/T/show_0Tw36PXMDfS6bsdktKsb7P/7hDGzZmowiabjBXSQPhZtl.json,"We'll see you next week and you know, wait for all the people to stop listening right here."
4,4514139,0.912559,data/raw/spotify-podcasts-2020/podcasts-transcripts/4/6/show_46XtTvRQDgGQP6Ig5FsZmc/6bwMqrBO8UQrEBV2YoCY1o.json,"First of all, that's a great name."
5,2994290,0.891342,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/3/show_03SdJMxJmYCgjHFSOdqepH/1ijRdyLKKgpVieJXR5bTCj.json,Or you can find it on my YouTube channel.
6,924887,0.850962,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/T/show_0tA6PeIYeet9Zfa0UrjzYi/2PfRZ6apY66BmQzVyR5tef.json,"All right, so bright colors are known to trigger our psyche and engage our intuitive minds and I wanted to ask you about that correlation, you know, even in the earliest decks such as the Mars."
7,3684466,0.854261,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/O/show_0OHMYcZChEVgTx2jz0yzze/7G88o9d8hN0vH1t313kGEz.json,Davenport theater features new stories nearly every Saturday and we bring the kid authors up on stage after the show for a live interview.
8,2362165,0.907928,data/raw/spotify-podcasts-2020/podcasts-transcripts/0/9/show_09KqYYMvpg3x43HQbZGIjE/4avQdo4HqQPS80t4crMWCE.json,I was ready to be like my man Mitch the Titans fan and get and get my luggage cooked a to you when we get off this podcast.
9,4472040,0.876805,data/raw/spotify-podcasts-2020/podcasts-transcripts/4/G/show_4gqIToEF1aYynAALHcUrma/4P4kcOfdS2tCoxEjC6niP9.json,This is what I believe the greatest eyewitness.


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

NOTE the model selection is overridden below to use the goemotion outout model."monologg/bert-base-cased-goemotions-original"  it requires additional flags to convert it from a pytorch model to tf.


In [ ]:
#model_checkpoint = "distilroberta-base"
#model_checkpoint = "monologg/bert-base-cased-goemotions-original"
model_checkpoint = "bert-base-uncased"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked. Don't panic about the warnings about inputs being too long for the model - remember that we'll be breaking them into shorter chunks right afterwards!

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["string"])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#tokenizer = AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original", from_pt=True)
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=['Unnamed: 0', 'confidence', 'source', 'string']
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

And now, we group texts together and chunk them into samples of length `block_size`. You can skip this step if your dataset is composed of individual sentences.

Now for the harder part: We need to concatenate all our texts together, and then split the result into chunks of a fixed size, which we will call block_size. To do this, we will use the map method again, with the option batched=True. When we use batched=True, the function we pass to map() will be passed multiple inputs at once, allowing us to group them into more or fewer examples than we had in the input. This allows us to create our new fixed-length samples.

We can use any block_size up to the the maximum length our model was pretrained with, which for models in the gpt2 family is usually something in the range 512-1024. This might be a bit too big to fit in your GPU RAM, though, so let's use something a bit smaller: 128.


In [ ]:

# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Note that we duplicate the inputs for our labels, without masking them. This is because CausalLM models in the 🤗 Transformers library automatically apply masking to the inputs, so we don't need to do it manually.

Also note that by default, the map method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of block_size every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower).

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import TFAutoModelForMaskedLM

#model = TFAutoModelForMaskedLM.from_pretrained("monologg/bert-base-cased-goemotions-original", from_pt=True)
#model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
model =  TFAutoModelForMaskedLM.from_pretrained("/content/drive/My Drive/Colab Notebooks/266-NLP-data/bert_pretrained_20000_13epoch")
#model =  TFAutoModelForMaskedLM.from_pretrained("/content/drive/My Drive/Colab Notebooks/266-NLP-data/goEmotion_pretrained_500000_2epoch")


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


We redefine our `optimizer` as we did with the CLM model, and we compile the model. We're using the internal loss again, like we did before.

In [ ]:
from transformers import create_optimizer, AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(lr=2e-5, weight_decay_rate=0.01)

model.compile(optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible for taking the samples and batching them in tensors.  Here we want to randomly mask tokens. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking. Note that our data collators are designed to work for multiple frameworks, so ensure you set the `return_tensors='tf'` argument to get Tensorflow tensors out - you don't want to accidentally get a load of `torch.Tensor` objects in the middle of your nice TF code!

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf"
)

Now we generate our datasets as before. Remember to pass the `data_collator` you just made to the `collate_fn` argument.

In [ ]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

test_set = lm_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Check pre fit and then, And now we fit our model! 

In [ ]:
# Get fit before pretraining
eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")

24/24 [==============================] - 13s 158ms/step - loss: 2.9277
Perplexity: 18.68


In [ ]:



model.fit(train_set, validation_data=validation_set, epochs=13)

Epoch 1/13
236/236 [==============================] - 136s 506ms/step - loss: 2.5587 - val_loss: 2.4115
Epoch 2/13
236/236 [==============================] - 121s 512ms/step - loss: 2.4306 - val_loss: 2.3887
Epoch 3/13
236/236 [==============================] - 122s 516ms/step - loss: 2.3639 - val_loss: 2.2698
Epoch 4/13
236/236 [==============================] - 123s 521ms/step - loss: 2.3156 - val_loss: 2.3973
Epoch 5/13
236/236 [==============================] - 122s 517ms/step - loss: 2.2764 - val_loss: 2.3574
Epoch 6/13
236/236 [==============================] - 122s 517ms/step - loss: 2.2633 - val_loss: 2.3083
Epoch 7/13
236/236 [==============================] - 123s 521ms/step - loss: 2.2201 - val_loss: 2.3383
Epoch 8/13
236/236 [==============================] - 122s 518ms/step - loss: 2.1859 - val_loss: 2.3139
Epoch 9/13
 76/236 [========>.....................] - ETA: 1:19 - loss: 2.1367

Like before, we can evaluate our model on the validation set and compute perplexity. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:


eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")

# Save model

In [ ]:


drive.mount('/content/drive',force_remount=True)

model.save_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_20000_13epoch")

drive.flush_and_unmount()

# Run with 500K

Load cached model

In [ ]:

model =  TFAutoModelForMaskedLM.from_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_20000_13epoch")
model.compile(optimizer=optimizer)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at /content/drive/My Drive/266-NLP-data/bert_pretrained_20000_13epoch.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


Load large corpus

In [ ]:
from google.colab import drive  
drive.mount('/content/drive', force_remount=True)
datasets = load_dataset("csv", 
                         data_files={"train": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_20000_with_context.csv",
                                     "validation": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_2000_with_context_validation.csv",
                                     "test": "/content/drive/My Drive/266-NLP-data/podcast_sentences_high_conf_2000_with_context_test.csv"})

tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=['Unnamed: 0', 'confidence', 'source', 'string']
)

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

test_set = lm_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Using custom data configuration default-37fd03610a554aa2
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


Mounted at /content/drive


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-fc8b0c5780097e8c.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-5c98d16e059f3bf0.arrow


...

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3ff6a8101b108817.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-d72179aa1f0b98ae.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-933c37d0653e55e4.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-4042329a37237fbf.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-37fd03610a554aa2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-8cbef88646cbd3a4.arrow


Get pre-training fit

In [ ]:
eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")

24/24 [==============================] - 13s 161ms/step - loss: 2.3060
Perplexity: 10.03


Run first epoch

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=1)
eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/266-NLP-data/bert_pretrained_500000_1epoch")

24/24 [==============================] - 4s 168ms/step - loss: 2.3482
Perplexity: 10.47


In [ ]:

model.save_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_500000_1epoch")

## Train second epoch


In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=1)
eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")
model.save_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_500000_2epoch")

24/24 [==============================] - 4s 170ms/step - loss: 2.2688
Perplexity: 9.67


In [ ]:

model.save_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_500000_2epoch")

Do a couple more?

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=8)
eval_results = model.evaluate(test_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")
model.save_pretrained("/content/drive/My Drive/266-NLP-data/bert_pretrained_500000_10epoch")

Epoch 1/8
236/236 [==============================] - 126s 533ms/step - loss: 2.0133 - val_loss: 2.3487
Epoch 2/8
236/236 [==============================] - 126s 532ms/step - loss: 1.9812 - val_loss: 2.2206
Epoch 3/8
236/236 [==============================] - 126s 532ms/step - loss: 1.9734 - val_loss: 2.3470
Epoch 4/8
236/236 [==============================] - 126s 532ms/step - loss: 1.9430 - val_loss: 2.2594
Epoch 5/8
236/236 [==============================] - 126s 532ms/step - loss: 1.9323 - val_loss: 2.3399
Epoch 6/8
236/236 [==============================] - 126s 532ms/step - loss: 1.8889 - val_loss: 2.2720
Epoch 7/8
236/236 [==============================] - 125s 532ms/step - loss: 1.8953 - val_loss: 2.2992
Epoch 8/8
24/24 [==============================] - 4s 173ms/step - loss: 2.3052
Perplexity: 10.03
